In [12]:
import pandas as pd
import numpy as np
import yfinance as yf

def calculate_atr(high, low, close, period=14):
    tr = np.maximum(high - low, np.abs(high - close.shift(1)), np.abs(low - close.shift(1)))
    atr = tr.rolling(window=period).mean()
    return atr

def backtest_atr_strategy(symbol, start_date, end_date, atr_period, atr_multiplier, ma_period, trailing_stop_atr_multiple, leverage=1): 
    # Fetch historical data
    df = yf.Ticker(symbol).history(start=start_date, end=end_date, interval='15m')
    #df = yf.Ticker(symbol).history(start=start_date, end=end_date)
    
    # Calculate ATR and Moving Average
    df['ATR'] = calculate_atr(df['High'], df['Low'], df['Close'], atr_period)
    df['MA'] = df['Close'].rolling(window=ma_period).mean()
    
    # Calculate ATR Stop and Trailing Stop
    df['ATR_Stop'] = df['Close'] - atr_multiplier * df['ATR']
    df['Trailing_Stop'] = df['Close'].cummax() - trailing_stop_atr_multiple * df['ATR']
    
    # Adjust ATR_Stop based on previous values
    for i in range(1, len(df)):
        if df['Close'].iloc[i] > df['ATR_Stop'].iloc[i - 1]:
            df.loc[df.index[i], 'ATR_Stop'] = max(
                df['Close'].iloc[i] - atr_multiplier * df['ATR'].iloc[i],
                df['ATR_Stop'].iloc[i - 1],
            )
        elif df['Close'].iloc[i] < df['ATR_Stop'].iloc[i - 1]:
            df.loc[df.index[i], 'ATR_Stop'] = min(
                df['Close'].iloc[i] + atr_multiplier * df['ATR'].iloc[i],
                df['ATR_Stop'].iloc[i - 1],
            )
    
    # Generate trading signals
    df['Signal'] = np.where(
        (df['Close'] > df['ATR_Stop']) &
        (df['Close'] > df['MA']) &
        (df['Close'] > df['Trailing_Stop']),
        1,
        0,
    )
    df['Position'] = df['Signal'].diff()
    df['Entry_Price'] = 0.0
    df['Trade_Active'] = False
    df['Trade_Result'] = 0.0
    
    # Execute trades based on signals
    for i in range(1, len(df)):
        if df['Position'].iloc[i] == 1:  # Entry signal
            df.loc[df.index[i], 'Entry_Price'] = float(df['Close'].iloc[i])
            df.loc[df.index[i], 'Trade_Active'] = True
        elif df['Position'].iloc[i] == -1 and df['Trade_Active'].iloc[i - 1]:
            df.loc[df.index[i], 'Trade_Result'] = (
                df['Close'].iloc[i] - df['Entry_Price'].iloc[i - 1]
            ) / df['Entry_Price'].iloc[i - 1]
            df.loc[df.index[i], 'Trade_Active'] = False
    
    # Apply leverage to trade results
    df['Strategy_Returns'] = df['Trade_Result'] * df['Position'] * leverage
    
    # Calculate asset returns
    df['Asset_Returns'] = df['Close'].pct_change()
    
    # Calculate Sharpe Ratios
    strategy_sharpe_ratio = (
        np.sqrt(252) * df['Strategy_Returns'].mean() / df['Strategy_Returns'].std()
    )
    
    if df['Asset_Returns'].std() == 0:
        asset_sharpe_ratio = 0 
    else:
        asset_sharpe_ratio = (
            np.sqrt(252) * df['Asset_Returns'].mean() / df['Asset_Returns'].std()
        )
    
    # Calculate cumulative returns
    df['Asset_Cumulative_Returns'] = (1 + df['Asset_Returns']).cumprod()
    df['Strategy_Cumulative_Returns'] = (1 + df['Strategy_Returns']).cumprod()
    
    # Total returns
    strategy_total_return = df['Strategy_Cumulative_Returns'].iloc[-1] - 1
    asset_total_return = df['Asset_Cumulative_Returns'].iloc[-1] - 1
    
    # Maximum drawdowns
    strategy_max_drawdown = (
        df['Strategy_Cumulative_Returns'] / df['Strategy_Cumulative_Returns'].cummax() - 1
    ).min()
    asset_max_drawdown = (
        df['Asset_Cumulative_Returns'] / df['Asset_Cumulative_Returns'].cummax() - 1
    ).min()
    
    return {
        'Strategy Total Return': strategy_total_return,
        'Asset Total Return': asset_total_return,
        'Strategy Sharpe Ratio': strategy_sharpe_ratio,
        'Asset Sharpe Ratio': asset_sharpe_ratio,
        'Strategy Max Drawdown': strategy_max_drawdown,
        'Asset Max Drawdown': asset_max_drawdown,
    }

def run_fixed_strategy(symbols, start_date, end_date, atr_period=14, atr_multiplier=1.5, ma_period=20, trailing_stop_atr_multiple=4, leverage=1):
    results = {}
    for symbol in symbols:
        print(f"Running backtest for {symbol} with fixed parameters...")
        perf = backtest_atr_strategy(
            symbol,
            start_date,
            end_date,
            atr_period,
            atr_multiplier,
            ma_period,
            trailing_stop_atr_multiple,
            leverage
        )
        results[symbol] = perf
    return results

if __name__ == "__main__":
    # Define parameters
    symbols = ['EURGBP=X']  # You can add more symbols if needed
    start_date = '2024-09-06'
    end_date = '2024-10-05'
    
    # Fixed strategy parameters
    atr_period = 14
    atr_multiplier = 0.5
    ma_period = 20
    trailing_stop_atr_multiple = 4
    leverage = 1  # Adjust leverage if needed
    
    # Run backtest with fixed parameters
    strategy_results = run_fixed_strategy(
        symbols,
        start_date,
        end_date,
        atr_period,
        atr_multiplier,
        ma_period,
        trailing_stop_atr_multiple,
        leverage
    )
    
    # Display the results
    for symbol, params in strategy_results.items():
        print(f"\nResults for {symbol}:")
        print(f"Strategy Total Return: {params['Strategy Total Return']:.2%}")
        print(f"Buy-and-Hold Total Return: {params['Asset Total Return']:.2%}")
        print(f"Strategy Sharpe Ratio: {params['Strategy Sharpe Ratio']:.2f}")
        print(f"Buy-and-Hold Sharpe Ratio: {params['Asset Sharpe Ratio']:.2f}")
        print(f"Strategy Max Drawdown: {params['Strategy Max Drawdown']:.2%}")
        print(f"Buy-and-Hold Max Drawdown: {params['Asset Max Drawdown']:.2%}")


Running backtest for EURGBP=X with fixed parameters...

Results for EURGBP=X:
Strategy Total Return: 0.28%
Buy-and-Hold Total Return: -0.76%
Strategy Sharpe Ratio: 0.99
Buy-and-Hold Sharpe Ratio: -0.21
Strategy Max Drawdown: 0.00%
Buy-and-Hold Max Drawdown: -1.77%


In [ ]:
# Fetch historical data function
def fetch_yfinance_data(symbol, interval, start_date, end_date):
    ticker = yf.Ticker(symbol)
    df = ticker.history(start=start_date, end=end_date, interval=interval)
    df.reset_index(inplace=True)
    if 'Datetime' in df.columns:
        df.rename(columns={'Datetime': 'Date'}, inplace=True)
    return df

# Function to calculate SMA and EMA
def calculate_sma_ema(df, sma_period=50, ema_period=20):
    if df.empty:
        return df
    df['SMA'] = df['Close'].rolling(window=sma_period).mean()
    df['EMA'] = df['Close'].ewm(span=ema_period, adjust=False).mean()
    return df

# Function to implement the trading strategy
def backtest_sma_ema_strategy(df):
    if df.empty:
        return pd.DataFrame()
    
    trades = []
    position = None  # No position initially

    for i in range(1, len(df)):
        today = df.iloc[i]
        yesterday = df.iloc[i - 1]

        # Ensure SMA and EMA are not NaN
        if pd.isna(yesterday['SMA']) or pd.isna(yesterday['EMA']):
            continue

        # Entry Conditions
        if position is None:
            # Long Entry: EMA crosses above SMA
            if yesterday['EMA'] <= yesterday['SMA'] and today['EMA'] > today['SMA']:
                entry_price = today['Open']
                position = {
                    'Entry_Time': today['Date'],
                    'Entry_Price': entry_price,
                    'Exit_Time': None,
                    'Exit_Price': None,
                    'Result': None,
                    'Type': 'Long'
                }
                trades.append(position)

            # Short Entry: EMA crosses below SMA
            elif yesterday['EMA'] >= yesterday['SMA'] and today['EMA'] < today['SMA']:
                entry_price = today['Open']
                position = {
                    'Entry_Time': today['Date'],
                    'Entry_Price': entry_price,
                    'Exit_Time': None,
                    'Exit_Price': None,
                    'Result': None,
                    'Type': 'Short'
                }
                trades.append(position)

        # Exit Conditions
        elif position is not None:
            # Long Exit: EMA crosses below SMA
            if position['Type'] == 'Long' and yesterday['EMA'] >= yesterday['SMA'] and today['EMA'] < today['SMA']:
                exit_price = today['Open']
                position['Exit_Time'] = today['Date']
                position['Exit_Price'] = exit_price
                position['Result'] = 'Win' if exit_price > position['Entry_Price'] else 'Loss'
                position = None

            # Short Exit: EMA crosses above SMA
            elif position['Type'] == 'Short' and yesterday['EMA'] <= yesterday['SMA'] and today['EMA'] > today['SMA']:
                exit_price = today['Open']
                position['Exit_Time'] = today['Date']
                position['Exit_Price'] = exit_price
                position['Result'] = 'Win' if exit_price < position['Entry_Price'] else 'Loss'
                position = None

    # Close any open positions at the end
    if position is not None and position['Exit_Time'] is None:
        last_row = df.iloc[-1]
        position['Exit_Time'] = last_row['Date']
        position['Exit_Price'] = last_row['Close']
        if position['Type'] == 'Long':
            position['Result'] = 'Win' if position['Exit_Price'] > position['Entry_Price'] else 'Loss'
        else:  # Short
            position['Result'] = 'Win' if position['Exit_Price'] < position['Entry_Price'] else 'Loss'
        position = None

    trades_df = pd.DataFrame(trades)
    return trades_df

# Function to calculate performance metrics
def calculate_sma_ema_metrics(df, trades_df):
    buy_and_hold_return = (df['Close'].iloc[-1] - df['Close'].iloc[0]) / df['Close'].iloc[0] * 100

    strategy_returns = trades_df.apply(
        lambda x: (x['Exit_Price'] - x['Entry_Price']) / x['Entry_Price']
        if x['Type'] == 'Long'
        else (x['Entry_Price'] - x['Exit_Price']) / x['Entry_Price']
        if x['Type'] == 'Short'
        else 0, axis=1
    )
    strategy_return = strategy_returns.sum() * 100

    cumulative_strategy_returns = (1 + strategy_returns).cumprod()
    peak_strategy = cumulative_strategy_returns.expanding(min_periods=1).max()
    drawdown_strategy = (cumulative_strategy_returns / peak_strategy) - 1
    max_drawdown_strategy = drawdown_strategy.min() * 100

    cumulative_bnh_returns = (1 + df['Close'].pct_change().fillna(0)).cumprod()
    peak_bnh = cumulative_bnh_returns.expanding(min_periods=1).max()
    drawdown_bnh = (cumulative_bnh_returns / peak_bnh) - 1
    max_drawdown_bnh = drawdown_bnh.min() * 100

    num_trades = len(trades_df)

    win_rate = len(trades_df[trades_df['Result'] == 'Win']) / num_trades * 100 if num_trades > 0 else 0

    avg_win = strategy_returns[trades_df['Result'] == 'Win'].mean() * 100 if len(strategy_returns[trades_df['Result'] == 'Win']) > 0 else 0
    avg_loss = strategy_returns[trades_df['Result'] == 'Loss'].mean() * 100 if len(strategy_returns[trades_df['Result'] == 'Loss']) > 0 else 0

    total_profit = strategy_returns[trades_df['Result'] == 'Win'].sum()
    total_loss = abs(strategy_returns[trades_df['Result'] == 'Loss'].sum())
    profit_factor = total_profit / total_loss if total_loss != 0 else float('inf')

    return {
        'Buy and Hold Return (%)': round(buy_and_hold_return, 2),
        'Max Drawdown Buy and Hold (%)': round(max_drawdown_bnh, 2),
        'Strategy Return (%)': round(strategy_return, 2),
        'Max Drawdown Strategy (%)': round(max_drawdown_strategy, 2),
        'Number of Trades': num_trades,
        'Win Rate (%)': round(win_rate, 2),
        'Average Win (%)': round(avg_win, 2),
        'Average Loss (%)': round(avg_loss, 2),
        'Profit Factor': round(profit_factor, 2)
    }

# Function to plot the trades
def plot_trades_sma_ema(df, trades_df, title):
    if df.empty:
        return

    plt.figure(figsize=(14, 7))
    plt.plot(df['Date'], df['Close'], label='Close Price', color='blue')
    plt.plot(df['Date'], df['SMA'], label='SMA', color='red', linestyle='--')
    plt.plot(df['Date'], df['EMA'], label='EMA', color='green', linestyle='--')

    # Plot Buy and Sell Signals
    buys = trades_df[trades_df['Type'] == 'Long']
    sells = trades_df[trades_df['Type'] == 'Short']

    plt.scatter(buys['Entry_Time'], buys['Entry_Price'], marker='^', color='green', label='Long Entry', zorder=5)
    plt.scatter(buys['Exit_Time'], buys['Exit_Price'], marker='v', color='red', label='Long Exit', zorder=5)

    plt.scatter(sells['Entry_Time'], sells['Entry_Price'], marker='v', color='red', label='Short Entry', zorder=5)
    plt.scatter(sells['Exit_Time'], sells['Exit_Price'], marker='^', color='green', label='Short Exit', zorder=5)

    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
